In [1]:
import requests
import os

FSbrukar = os.environ["FSbrukar"]
FSpassord = os.environ["FSpassord"]

def query_FS_graphql(query, variable):
    hode = {
        'Accept': 'application/json;version=1',
        "Feature-Flags": "beta, experimental"
    }
    GraphQLurl = "https://api.fellesstudentsystem.no/graphql/"
    svar = requests.post(
        GraphQLurl, 
        json = {
            'query': query,
            'variables': variable
        },
        headers=hode,
        auth=(FSbrukar, FSpassord))
    if 200 <= svar.status_code < 300:
        return svar.json()
    else:
        return 


In [71]:
query_termin = """
query MyQuery($arstall: [Int!], $terminbetegnelse: [ID!]) {
  terminer(
    filter: {eierOrganisasjonskode: "0203", arstall: $arstall, terminbetegnelser: $terminbetegnelse}
  ) {
    nodes {
      id
      betegnelse {
        kode
      }
      arstall
    }
  }
}
"""


query_emneid = """
query MyQuery($emnekode: [String!]) {
  emner(filter: {eierInstitusjonsnummer: "0203", emnekoder: $emnekode}) {
    nodes {
      id
    }
  }
}
"""

query_vurderingsenheter = """
query MyQuery ($emne_id: [ID!], $termin_id: [ID!]) {
  vurderingsenheter(
    filter: {eierOrganisasjonskode: "203", emner: $emne_id, terminer: $termin_id}
  ) {
    nodes {
      vurderingsmeldinger {
        student {
          vurderingsforsokForEmner(emnekoder: {emnekode: "DAT102", versjonskode: "1"}) {
            antallTellendeForsok
          }
          studentnummer
        }
      }
    }
  }
}
"""

query_partiplassering = """
query MyQuery {
undervisningspartiplasseringer(
    filter: {eierOrganisasjonskode: "203", emneSkalEksporteresTilLms: true, undervisningsaktiviteter: ["MTIwOjIwMywyMDMsREFUMTAyLDEsMjAyNSxWw4VSLDEsMQ", "MTIwOjIwMywyMDMsREFUMTAyLDEsMjAyNSxWw4VSLDEsMw", "MTIwOjIwMywyMDMsREFUMTAyLDEsMjAyNSxWw4VSLDEsNQ", "MTIwOjIwMywyMDMsREFUMTAyLDEsMjAyNSxWw4VSLDEsNg" ]}
    first: 1000
  ) {
    nodes {
      student {
        studentnummer
      }
      undervisningsaktivitet {
        kode
      }
    }
  }
}
"""

In [10]:
variable = {'arstall': 2025, 'terminbetegnelse': "MTE3OjIwMyxWw4VS"}
svar = query_FS_graphql(query_termin, variable)
termin_id = svar['data']['terminer']['nodes'][0]['id']



In [15]:
emnekode = "DAT102"
variable = {'emnekode': emnekode, }
svar = query_FS_graphql(query_emneid, variable)
emne_id = svar['data']['emner']['nodes'][0]['id']



In [36]:
variable = {'emne_id': emne_id, 'termin': termin_id}
svar = query_FS_graphql(query_vurderingsenheter, variable)
vurderingsmeldinger = svar['data']['vurderingsenheter']['nodes']


In [72]:
svar = query_FS_graphql(query_partiplassering, variable)
partiplasseringer = svar['data']['undervisningspartiplasseringer']['nodes']
plassert = []
for p in partiplasseringer:
    plassert.append([p['student']['studentnummer'], p['undervisningsaktivitet']['kode']])


In [76]:
ekstra_vurderingsmeldt = []
for v in vurderingsmeldinger:
    if len(v['vurderingsmeldinger']) > 0:
        for s in v['vurderingsmeldinger']:
            if len(s['student']['vurderingsforsokForEmner']) > 0:
                # print(f"{s['student']['studentnummer']} - {s['student']['vurderingsforsokForEmner'][0]['antallTellendeForsok']}")
                ekstra_vurderingsmeldt.append(s['student']['studentnummer'])


In [78]:
import pandas as pd
data = pd.read_csv("enrollments.csv")


C:\Users\AKV\AppData\Local\Temp\ipykernel_11824\1635647140.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("enrollments.csv")


In [91]:
data = data.copy()
data.dropna(subset=["section_id"], inplace=True)

In [92]:

len(data[data['section_id'].str.startswith("UA_203_DAT102_1_2025_VÅR_1")])


225

In [93]:
data.head()

,course_id,user_id,role,section_id,status,associated_user_id
0,NaN,fs:203:600166,student,UA_203_MAT110_1_2025_VÅR_1_2,completed,NaN
1,NaN,fs:203:600166,student,UE_203_MAT110_1_2025_VÅR_1,completed,NaN
2,NaN,fs:203:209759,teacher,UE_203_Å-KH230_1_2026_VÅR_1,active,NaN
3,NaN,fs:203:714472,student,UE_203_Å-KH230_1_2025_VÅR_1,active,NaN
4,NaN,fs:203:638877,student,UE_203_Å-KH230_1_2025_VÅR_1,active,NaN
